In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')



In [2]:
#Elements nécessaires au web-scrapping
import requests
import random
from bs4 import BeautifulSoup
import re

In [3]:
#L'objectif est de récupérer dans un tableau les informations détaillées issues des dégustations de cafés
#réalisées par le site "www.coffeereview.com" . Il s'agit de la liste des "highest-rated-coffees" ,
#cafés dont la  cotation est >= 94 ( sur 100)


#L'extraction des données se fera en deux temps:

#Tout d'abord : la liste des adresses renvoyant aux comptes rendus détaillés des dégustations 
#en parcourant les 85 pages accessibles à partir de l'adresse' "https://www.coffeereview.com/highest-rated-coffees/"


#Ensuite dans une grande boucle , les informations détaillées de chaque dégustation

In [4]:
#Récupération des adresses des différentes pages , 1 page par café testé

#NB : contournement de l'erreur 403 qui bloque l'accès au site en webscraping
#Solution issue du site : https://scrapeops.io/web-scraping-playbook/403-forbidden-error-web-scraping/

#Première page
pages_revues=[]
user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
]

r = requests.get("https://www.coffeereview.com/highest-rated-coffees/", headers={'User-Agent': random.choice(user_agents_list)})
pages_revues.append(BeautifulSoup(r.content,'html.parser'))

#Pages suivantes de 2 à 85

for i in range(2,86):
    adresse="https://www.coffeereview.com/highest-rated-coffees/page/"+str(i)+"/"
    r = requests.get(adresse, headers={'User-Agent': random.choice(user_agents_list)})
    pages_revues.append(BeautifulSoup(r.content,'html.parser'))

In [5]:
#Vérification du nombre de pages récupérées ( revue=review)
print ("Taille de la liste des pages :",len(pages_revues))

#Cela correspond bien à la numérotation des pages du site

Taille de la liste des pages : 85


In [6]:
#Constitution de la liste des liens vers les comptes rendus de dégustation
liens_revues=[]

for i in range(len(pages_revues)):
    liste_cafés=pages_revues[i].find_all(attrs={"class":"review-title"})
    for element in liste_cafés:
        lien=element.find("a")
        liens_revues.append(lien['href'])
    

In [7]:
#Vérification du nombre de liens récupérés , 1 par café testé
print("Nombre de liens récupérés:",len(liens_revues))    

Nombre de liens récupérés: 1700


In [8]:
#Quelques exemples de liens vers les comptes rendus
liens_revues[0:15]

['https://www.coffeereview.com/review/kenya-10/',
 'https://www.coffeereview.com/review/sl34-dark-roast/',
 'https://www.coffeereview.com/review/ethiopia-bench-maji-geisha-lucy-station-dark-roast/',
 'https://www.coffeereview.com/review/kenya-ab-athena-plus/',
 'https://www.coffeereview.com/review/kenya-muranga/',
 'https://www.coffeereview.com/review/costa-rica-turrialba-volcano-honey-process/',
 'https://www.coffeereview.com/review/colombia-huila-finca-el-triunfo-martha-zuniga-pink-bourbon/',
 'https://www.coffeereview.com/review/kenya-kanwangi-aa/',
 'https://www.coffeereview.com/review/ethiopia-bench-maji-geisha-lucy-station/',
 'https://www.coffeereview.com/review/ecuador-penaherrera-aaa-2/',
 'https://www.coffeereview.com/review/hwc-geisha-plus-geisha-black-espresso/',
 'https://www.coffeereview.com/review/ethiopia-yirgacheffe-jade-lotus/',
 'https://www.coffeereview.com/review/blue-sunda-estate-java/',
 'https://www.coffeereview.com/review/la-papaya-natural-ecuador/',
 'https://

In [9]:
#Il s'agit maintenant d'extraire toutes les informations des revues
#Cette opération est assez longue.
#Elle inclut un premier traitement de mise en forme des informations extraites
#Traitement mis au point à partir de quelques exemples


#Mesure du temps
heure_deb = dt.datetime.now()

#Intialisation des listes
café=[]
nom_torréfacteur=[]
cotation=[]
lieu_torréfaction=[]
origine_café=[]
niveau_torréfaction=[]
prix_estimatif=[]

date_test=[]
note_arome=[]
note_corps=[]
note_saveur=[]
note_arrière_gout=[]
note_avec_lait=[]
note_acidité_structure=[]

jugement=[]
notes=[]
conclusion=[]
nb_cafés=0

#Parcours de la liste de liens
for lien in liens_revues:
    
        #Initialisation des toutes les variables à NA
        nomcafé="NA"
        nomtorréfacteur="NA"
        notecotation="NA"
        lieutorréfaction="NA"
        originecafé="NA"
        niveautorréfaction="NA"
        prix="NA"
        datetest="NA"
        arome="NA"
        corps="NA"
        saveur="NA"
        arrièregout="NA"
        aveclait="NA"
        acidité="NA"
        
        #Boucle sur les cafés
        nb_cafés+=1
        r = requests.get(lien, headers={'User-Agent': random.choice(user_agents_list)})
        infos=BeautifulSoup(r.content,'html.parser')

        #Récupération des premières infos café , torréfacteur et cotation
        nomcafé=infos.find(name="h1",attrs={"class":"review-title"}).text
        nomtorréfacteur=infos.find(name="p",attrs={"class":"review-roaster"}).text
        notecotation=infos.find(attrs={"class":"review-template-rating"}).text

        café.append(nomcafé)
        nom_torréfacteur.append(nomtorréfacteur)
        cotation.append(notecotation)
        
        #Récupération des 2 tableaux
        tableau=infos.find_all(attrs={"class":"review-template-table"})
        tableau_1=re.compile(r"\n").sub(" ",tableau[0].getText().strip())
        tableau_2=re.compile(r"\n").sub(" ",tableau[1].getText().strip())
        tableau_2=re.compile(r"\t\t\t\t\t\t").sub(" ",tableau_2)
        liste_1=re.compile(r"   ").split(tableau_1)
        liste_2=re.compile(r"   ").split(tableau_2)
   
        #Décomposition de la liste 1
        lieu_torréfaction.append(liste_1[0].split(": ")[1])
        origine_café.append(liste_1[1].split(": ")[1])
        niveau_torréfaction.append(liste_1[2].split(": ")[1])
        if len(liste_1)>4:
            prix_estimatif.append(liste_1[4].split(": ")[1])
        else:  
            prix_estimatif.append("NA")

        #Décomposition de la liste 2
        
        if len(liste_2)>0:
            for i in range(len(liste_2)):
                test=liste_2[i].split(": ")
                if test[0]=="Review Date":
                    datetest=test[1]
                if test[0]=="Aroma":
                    arome=test[1]
                if test[0]=="Body":
                    corps=test[1]    
                if test[0]=="Flavor":
                    saveur=test[1]   
                if test[0]=="Aftertaste":
                    arrièregout=test[1]   
                if test[0]=="Acidity/Structure":
                    acidité=test[1]
                if test[0]=="With Milk":
                    aveclait=test[1]
        
        date_test.append(datetest)
        note_arome.append(arome)
        note_corps.append(corps)
        note_saveur.append(saveur)
        note_arrière_gout.append(arrièregout)
        note_avec_lait.append(aveclait)
        note_acidité_structure.append(acidité)
        
        #Récupération des textes
        textes=infos.find(attrs={"class":"review-template"}).getText().strip()

        #traitement pour découper le texte en 3 zones correspondant aux 3 rubriques de la revue
        textes=re.compile(r"\n").sub(" ",textes) # on éclairçit le texte
        textes= re.compile(r'\r').sub(" ",textes) 

        #On récupère les contenus en coupant sur les intitulés des rubriques

        if re.search("Bottom Line", textes):
            texte1=textes.split("Bottom Line ")
            texte11=texte1[1].split("Explore")
        elif re.search("Who Should Drink It", textes):
            texte1=textes.split("Who Should Drink It")
            texte11=texte1[1].split("Explore")
        else:
            texte11=["NA"]
        
        conclusion.append(texte11[0].strip())

        texte2=texte1[0].split("Notes ")
        texte21=texte2[1].split("Visit")
    
        notes.append(texte21[0].strip()) 
    
        texte3=texte2[0].split("Blind Assessment")
        jugement.append(texte3[1].strip())

    
heure_fin = dt.datetime.now() 

#Nombre de cafés traités
print("Nombre de cafés traités:",nb_cafés)

#Temps écoulé en minutes
temps=((heure_fin-heure_deb).seconds)/60

print("Temps de traitement en minutes :",temps)

Nombre de cafés traités: 1700
Temps de traitement en minutes : 21.583333333333332


In [10]:
#Construction du dataframe 

data_café=pd.DataFrame({
    "café":list(café),
    "nom_torréfacteur":list(nom_torréfacteur),
    "cotation":list(cotation),
    "date_test":list(date_test),
    "note_arôme":list(note_arome),
    "note_saveur":list(note_saveur),
    "note_corps":list(note_corps),
    "note_arrière_gout":list(note_arrière_gout),
    "note_acidité_structure": list(note_acidité_structure),
    "note_avec_lait":list(note_avec_lait),
    "lieu_torréfaction":list(lieu_torréfaction),
    "origine_café":list(origine_café),
    "niveau_torréfaction":list(niveau_torréfaction),
    "jugement":list(jugement),
    "notes":list(notes),
    "conclusion":list(conclusion) ,
    "prix_estimatif":list(prix_estimatif),
})

In [11]:
#Colonne origine_café à traiter pour enlever des points
data_café["origine_café"]=data_café["origine_café"].apply(lambda x:x.split(".")[0])

In [12]:
#Dernier traitement :Remplacer les NR par des valeurs manquantes
data_café=data_café.replace(to_replace=["NR"],value=["NA"])

In [13]:
#Exportation du fichier pour analyse subséquente
data_café.to_csv(r"C:\DATA\café_1\TOP-RATED_COFFEES.csv",index=False)